In [1]:
from __future__ import print_function

import numpy as np
from torch.utils import data
import argparse
import torch
import torch.utils.data as data_utils
import torch.optim as optim
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
import math
seed = 941027
torch.manual_seed(seed)
torch.cuda.manual_seed(0)
np.random.seed(seed)
#2021,7620,736,534
# torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)
    
    
class Dataset(data.Dataset):
    def __init__(self, X1,Y):
        self.X1 = X1
        self.Y = Y
    def __len__(self):        
        return len(self.X1)
    
    def __getitem__(self, index):
        x = self.X1[index]
        y = self.Y[index]
        return x,y

#from model import Attention, GatedAttention

## Negetive pictures

In [2]:
import os
def readFeatures(path):
    features=np.array([])
    start=0
    for file in os.listdir(path):
        d = os.path.join(path, file)
        datafile=d+"/features.npy"
        try:
            temp=np.load(datafile)
#             print(datafile)
#             print(temp.shape)
            if(start==0):
                features=temp
                if(d[-10]=='S'):
                    label=np.ones(len(temp))
                else:
                    label=np.zeros(len(temp))
                start=1
            else:
                features=np.vstack((features,temp))
                if(d[-10]=='S'):
                    label=np.append(label,np.ones(len(temp)))
                else:
                    label=np.append(label,np.zeros(len(temp)))
        except IOError:
            print('file not in Scripted')

            
    return features,label

def load_data(path,mode):
    if mode == "test":
        xTest,yTest=readFeatures(path+"/"+mode)
        xTest = xTest.reshape(len(xTest),1,19,8)
        return Dataset(xTest,yTest)
        
    elif mode == "train":
        xTrain,yTrain=readFeatures(path+"/"+mode)
        xTrain = xTrain.reshape(len(xTrain),1,19,8)
        return Dataset(xTrain,yTrain)
        
    elif mode == "val":
        xVal,yVal=readFeatures(path+"/"+mode)
        xVal = xVal.reshape(len(xVal),1,19,8)
        return Dataset(xVal,yVal)
    else:
        print("Mode not defined")
        return

traindataset = load_data("../Participant_wise_physio_change",'train')
testdataset = load_data("../Participant_wise_physio_change",'test')
valdataset = load_data("../Participant_wise_physio_change",'val')

In [3]:
class Stammering(data_utils.Dataset):
    def __init__(self, target_number=1, mean_bag_length=5, var_bag_length=2, num_bag=150, seed=2021, train="train"):
        self.target_number = target_number
        self.mean_bag_length = mean_bag_length
        self.var_bag_length = var_bag_length
        self.num_bag = num_bag
        self.train = train
        self.r = np.random.RandomState(seed)

        if self.train=="train":
            self.train_bags_list, self.train_labels_list = self._create_bags()
        elif self.train=="val":
            self.val_bags_list, self.val_labels_list = self._create_bags()
        else:
            self.test_bags_list, self.test_labels_list = self._create_bags()

    def _create_bags(self):
        if self.train=="train":
            print("train")
            loader = data_utils.DataLoader(traindataset,
                                           batch_size=1,
                                           shuffle=True)
        elif self.train=="val":
            print("val")
            loader = data_utils.DataLoader(valdataset,
                                           batch_size=1,
                                           shuffle=True)
        else:
            loader = data_utils.DataLoader(testdataset,
                                           batch_size=1,
                                           shuffle=True)
            
        bags_list = []
        labels_list = []
        for (batch_data, batch_labels) in loader:
            #print(batch_data.shape)
            bags_list.append(batch_data.reshape(19,1,8))
            temp = torch.as_tensor(np.array([batch_labels for x in range(19)]))
            labels_list.append(temp)
            
               
        #print(bags_list)
        #print(labels_list)
        return bags_list, labels_list

    def __len__(self):
        if self.train=="train":
            return len(self.train_labels_list)
        elif self.train=="val":
            return len(self.val_labels_list)
        else:
            return len(self.test_labels_list)

    def __getitem__(self, index):
        if self.train=="train":
            bag = self.train_bags_list[index]
            label = [max(self.train_labels_list[index]), self.train_labels_list[index]]
        elif self.train=="val":
            bag = self.val_bags_list[index]
            label = [max(self.val_labels_list[index]), self.val_labels_list[index]]
        else:
            bag = self.test_bags_list[index]
            label = [max(self.test_labels_list[index]), self.test_labels_list[index]]

        return bag, label

In [4]:
train_loader = data_utils.DataLoader(Stammering(train="train"),batch_size=1,worker_init_fn=seed_worker,shuffle=True)
test_loader = data_utils.DataLoader(Stammering(train="test"),batch_size=1,worker_init_fn=seed_worker,shuffle=True)
val_loader = data_utils.DataLoader(Stammering(train="val"),batch_size=1,worker_init_fn=seed_worker,shuffle=True)

train
val


HR 32 acc 58

HR 64 ACC 53

HR 32 EDA 32 acc 60

HR 32 EDA 16 RSPR 32 A 57

HR 32 EDA 32 RSPR 32 a 33

hr 32 eda 32 rspr 16 rspa 32 a 37

hr 64 eda 32 rspr 16 rsp 32 a 30

hr 32 eda 64 rspr 16 rspr 32 a 0

hr 64 eda 32 rspr 16 rspr 16 a 53

hr 64 eda 64 16 16 a 32 

hr 32 eda 32 rspr 16 rspa 16 d 256 acc 35


hr 32 eda 32 rspr 32 rspa 16 d 256 acc 61

same l 256

In [5]:
class _NonLocalBlockND(nn.Module):
    def __init__(self, in_channels, inter_channels=None, dimension=3, sub_sample=True, bn_layer=True):
        super(_NonLocalBlockND, self).__init__()

        assert dimension in [1, 2, 3]

        self.dimension = dimension
        self.sub_sample = sub_sample

        self.in_channels = in_channels
        self.inter_channels = inter_channels

        if self.inter_channels is None:
            self.inter_channels = in_channels // 2
            if self.inter_channels == 0:
                self.inter_channels = 1

        if dimension == 3:
            conv_nd = nn.Conv3d
            max_pool_layer = nn.MaxPool3d(kernel_size=(1, 2, 2))
            bn = nn.BatchNorm3d
        elif dimension == 2:
            conv_nd = nn.Conv2d
            max_pool_layer = nn.MaxPool2d(kernel_size=(2, 2))
            bn = nn.BatchNorm2d
        else:
            conv_nd = nn.Conv1d
            max_pool_layer = nn.MaxPool1d(kernel_size=(2))
            bn = nn.BatchNorm1d

        self.g = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                         kernel_size=1, stride=1, padding=0)

        if bn_layer:
            self.W = nn.Sequential(
                conv_nd(in_channels=self.inter_channels, out_channels=self.in_channels,
                        kernel_size=1, stride=1, padding=0),
                bn(self.in_channels)
            )
            nn.init.constant(self.W[1].weight, 0)
            nn.init.constant(self.W[1].bias, 0)
        else:
            self.W = conv_nd(in_channels=self.inter_channels, out_channels=self.in_channels,
                             kernel_size=1, stride=1, padding=0)
            nn.init.constant(self.W.weight, 0)
            nn.init.constant(self.W.bias, 0)

        self.theta = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                             kernel_size=1, stride=1, padding=0)
        self.phi = conv_nd(in_channels=self.in_channels, out_channels=self.inter_channels,
                           kernel_size=1, stride=1, padding=0)

        if sub_sample:
            self.g = nn.Sequential(self.g, max_pool_layer)
            self.phi = nn.Sequential(self.phi, max_pool_layer)

    def forward(self, x):
        '''
        :param x: (b, c, t, h, w)
        :return:
        '''

        batch_size = x.size(0)

        g_x = self.g(x).view(batch_size, self.inter_channels, -1)
        g_x = g_x.permute(0, 2, 1)

        theta_x = self.theta(x).view(batch_size, self.inter_channels, -1)
        theta_x = theta_x.permute(0, 2, 1)
        phi_x = self.phi(x).view(batch_size, self.inter_channels, -1)
        f = torch.matmul(theta_x, phi_x)
        f_div_C = F.softmax(f, dim=-1)

        y = torch.matmul(f_div_C, g_x)
        y = y.permute(0, 2, 1).contiguous()
        y = y.view(batch_size, self.inter_channels, *x.size()[2:])
        W_y = self.W(y)
        z = W_y + x

        return z


class NONLocalBlock1D(_NonLocalBlockND):
    def __init__(self, in_channels, inter_channels=None, sub_sample=False, bn_layer=True):
        super(NONLocalBlock1D, self).__init__(in_channels,
                                              inter_channels=inter_channels,
                                              dimension=1, sub_sample=sub_sample,
                                              bn_layer=bn_layer)
class NONLocalBlock2D(_NonLocalBlockND):
    def __init__(self, in_channels, inter_channels=None, sub_sample=True, bn_layer=True):
        super(NONLocalBlock2D, self).__init__(in_channels,
                                              inter_channels=inter_channels,
                                              dimension=2, sub_sample=sub_sample,
                                              bn_layer=bn_layer)

# class NonLocalBlock(nn.Module):
#     def __init__(self, channel):
#         super(NonLocalBlock, self).__init__()
#         self.inter_channel = channel
#         self.conv_phi = nn.Conv2d(in_channels=channel, out_channels=self.inter_channel, kernel_size=1, stride=1,padding=0, bias=False)
#         self.conv_theta = nn.Conv2d(in_channels=channel, out_channels=self.inter_channel, kernel_size=1, stride=1, padding=0, bias=False)
#         self.conv_g = nn.Conv2d(in_channels=channel, out_channels=self.inter_channel, kernel_size=1, stride=1, padding=0, bias=False)
#         self.softmax = nn.Softmax(dim=1)
#         self.conv_mask = nn.Conv2d(in_channels=self.inter_channel, out_channels=channel, kernel_size=1, stride=1, padding=0, bias=False)

#     def forward(self, x):
#         # [N, C, H , W]
#         b, c, h, w = x.size()
#         # [N, C/2, H * W]
# #         print(b, c, h, w)
# #         print(self.conv_phi(x).view(b, c, -1).shape)
#         x_phi = self.conv_phi(x).view(b, c, -1)
#         # [N, H * W, C/2]
#         x_theta = self.conv_theta(x).view(b, c, -1).permute(0, 2, 1).contiguous()
#         x_g = self.conv_g(x).view(b, c, -1).permute(0, 2, 1).contiguous()
#         # [N, H * W, H * W]
#         mul_theta_phi = torch.matmul(x_theta, x_phi)
#         mul_theta_phi = self.softmax(mul_theta_phi)
#         # [N, H * W, C/2]
#         mul_theta_phi_g = torch.matmul(mul_theta_phi, x_g)
#         # [N, C/2, H, W]
#         mul_theta_phi_g = mul_theta_phi_g.permute(0,2,1).contiguous().view(b,self.inter_channel, h, w)
#         # [N, C, H , W]
#         mask = self.conv_mask(mul_theta_phi_g)
#         out = mask + x
#         return out

class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        self.L = 128
        self.D = 256
        self.K = 1

#         self.nl = NonLocalBlock(4)
        self.nl=NONLocalBlock1D(4)

        self.lossfn=nn.MSELoss()
        
        self.attention_transform_M = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.L*self.K*4, 512),
            nn.Tanh(),
            nn.Linear(512,self.K)
        )

        self.feature_extractor_HR = nn.Sequential(
#             nn.Linear(2,64),
#             nn.ReLU(),
            nn.Linear(2,self.D),
            nn.ReLU()
        )
        self.feature_extractor_EDA = nn.Sequential(
#             nn.Linear(2,32),
#             nn.ReLU(),
            nn.Linear(2,self.D),
            nn.ReLU()
        )
        self.feature_extractor_RSPR= nn.Sequential(
#             nn.Linear(2,16),
#             nn.ReLU(),
            nn.Linear(2,self.D),
            nn.ReLU()
        )
        self.feature_extractor_RSPA= nn.Sequential(
#             nn.Linear(2,64),
#             nn.ReLU(),
            nn.Linear(2,self.D),
            nn.ReLU()
        )
        
        self.feature_transform_HR = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(self.D, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True),
            nn.Linear(self.D, self.L)
        )
        
        self.attention_transform_HR = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.D, self.L),
            nn.Tanh(),
            nn.Linear(self.L,self.K)
        )
        
        self.feature_transform_EDA = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(self.D, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True),
            nn.Linear(self.D, self.L)
        )
        
        self.attention_transform_EDA = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.D, self.L),
            nn.Tanh(),
            nn.Linear(self.L,self.K)
        )
        
        self.feature_transform_RSPA = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(self.D, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True),
            nn.Linear(self.D, self.L)
        )
        
        self.attention_transform_RSPA = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.D, self.L),
            nn.Tanh(),
            nn.Linear(self.L,self.K)
        )
        
        self.feature_transform_RSPR = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.BatchNorm1d(self.D, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True),
            nn.Linear(self.D, self.L)
        )
        
        self.attention_transform_RSPR = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.D, self.L),
            nn.Tanh(),
            nn.Linear(self.L,self.K)
        )

        self.classifier = nn.Sequential(
            nn.Linear(self.L*self.K*4, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
        
    def attention_pooling(self,feature_embed_HR,feature_embed_EDA,feature_embed_RSPA,feature_embed_RSPR):
        
        #HR
        feature_transform_HR = self.feature_transform_HR(feature_embed_HR)
        
        A_t_hr = self.attention_transform_HR(feature_embed_HR)
        
        A_t_hr = torch.transpose(A_t_hr, 1, 0)
        
        A_t_hr = F.softmax(A_t_hr, dim=1)

        M_hr = torch.mm(A_t_hr, feature_transform_HR)
        
        #EDA
        
        feature_transform_EDA = self.feature_transform_HR(feature_embed_EDA)
        
        A_t_eda = self.attention_transform_EDA(feature_embed_EDA)
        
        A_t_eda = torch.transpose(A_t_eda, 1, 0)
        
        A_t_eda = F.softmax(A_t_eda, dim=1)

        M_eda = torch.mm(A_t_eda, feature_transform_EDA)
        
        #RSPA

        feature_transform_RSPA = self.feature_transform_RSPA(feature_embed_RSPA)
        
        A_t_rspa = self.attention_transform_RSPA(feature_embed_RSPA)
        
        A_t_rspa = torch.transpose(A_t_rspa, 1, 0)
        
        A_t_rspa = F.softmax(A_t_rspa, dim=1)

        M_rspa = torch.mm(A_t_rspa, feature_transform_RSPA)
        
        #RSPR

        feature_transform_RSPR = self.feature_transform_RSPR(feature_embed_RSPR)
        
        A_t_rspr = self.attention_transform_RSPR(feature_embed_RSPR)
        
        A_t_rspr = torch.transpose(A_t_rspr, 1, 0)
        
        A_t_rspr = F.softmax(A_t_rspr, dim=1)

        M_rspr = torch.mm(A_t_rspr, feature_transform_RSPR)
        
        M=torch.cat((M_hr,M_eda,M_rspa,M_rspr),dim=1)
        
        return M
        

    def forward(self, x):
        
        
        x=x.squeeze(0)
        
        #HR
        
        feature_embed_HR= self.feature_extractor_HR(x[:,:,:2])
        

        
        #EDA
        
        feature_embed_EDA= self.feature_extractor_EDA(x[:,:,2:4])
        
        
        #RSPA
        
        feature_embed_RSPA= self.feature_extractor_RSPA(x[:,:,4:6])
        
        
        #RSPR
        
        feature_embed_RSPR= self.feature_extractor_RSPR(x[:,:,6:8])
        
        
        
        M = self.attention_pooling(feature_embed_HR,feature_embed_EDA,feature_embed_RSPA,feature_embed_RSPR)
        
#         print(M.shape)
        
        M = M.reshape(1,4,self.L)
        
#         print(M.shape)
        
        M = self.nl(M) 
        
#         print(M.shape)
        
        M=M.reshape(M.size(0),-1)
        
#         print(M.shape)

        Y_prob = self.classifier(M)        
        
        Y_hat = torch.ge(Y_prob, 0.5).float()

        return Y_prob, Y_hat



    # AUXILIARY METHODS
    def calculate_classification_error(self, X, Y):
        Y = Y.float()
        _, Y_hat = self.forward(X)
        error = 1. - Y_hat.eq(Y).cpu().float().mean().item()

        return error, Y_hat

    def calculate_objective(self, X, Y):
        Y = Y.float()
        Y_prob, _ = self.forward(X)
        Y_prob = torch.clamp(Y_prob, min=1e-5, max=1. - 1e-5)
        loss=self.lossfn(Y_prob.squeeze(1),Y)
        
        return loss

In [6]:

# model.cuda()

In [7]:
# from torchsummary import summary
# summary(model.cuda(),(1,24))

In [8]:


model = Attention()
optimizer = optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999))
if torch.cuda.is_available():model.cuda()

best_model = "./saved_models/best_model_MIL_raw_cs_nl"
train_acc = []
modelname=[]
val_acc = []
best_acc = 0
for epoch in range(0, 50): #28-2021
    model.train()
    train_loss = 0.
    train_error = 0.
    y =[]
    ypred = []
    for batch_idx, (data, label) in enumerate(train_loader):
        bag_label = label[0]
        data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)
        optimizer.zero_grad()
        loss = model.calculate_objective(data.float(), bag_label.float())
        train_loss += loss.item()
        error,y_pred = model.calculate_classification_error(data.float(), bag_label.float())
        train_error += error
        y_pred=y_pred.squeeze(1)
        ypred.extend(y_pred.tolist())
        y.extend(bag_label.tolist())
        loss.backward()
        optimizer.step()
    
    trainacc = f1_score(y,ypred)
    train_loss /= len(train_loader)
    train_error /= len(train_loader)
    print("EPOCH ",epoch)
    print('Train : Loss: {:.4f}, Train error: {:.4f}, Train acc : {}'.format(train_loss, 
                                                                                train_error,trainacc))
    y =[]
    ypred = []
    model.eval()
    for batch_idx, (data, label) in enumerate(val_loader):
        bag_label = label[0]
        data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)
        error, y_pred = model.calculate_classification_error(data.float(), bag_label.float())
        y_pred=y_pred.squeeze(1)
        ypred.extend(y_pred.tolist())
        y.extend(bag_label.tolist())
    valacc = f1_score(y,ypred)
    print('Val acc :{}'.format(valacc))
    if (valacc>=best_acc and trainacc>0.65):
        print("---------State saved---------")
        best_acc = valacc
        best_state=model.state_dict()
        torch.save(best_state, best_model+'_epoch_'+str(epoch)+".pth")
        modelname.append(best_model+'_epoch_'+str(epoch)+".pth")
    print('Best validation accuracy ',best_acc)

    y =[]
    ypred = []
    for batch_idx, (data, label) in enumerate(test_loader):
        bag_label = label[0]
        data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)
        error, y_pred = model.calculate_classification_error(data.float(), bag_label.float())
        y_pred=y_pred.squeeze(1)
        ypred.extend(y_pred.tolist())
        y.extend(bag_label.tolist())
    valacc = f1_score(y,ypred)
    print('test acc :{}'.format(valacc))
        

/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.


EPOCH  0
Train : Loss: 0.2467, Train error: 0.4574, Train acc : 0.3723404255319149
Val acc :0.5576923076923076
Best validation accuracy  0
test acc :0.0
EPOCH  1
Train : Loss: 0.2314, Train error: 0.3740, Train acc : 0.42042042042042044
Val acc :0.6573426573426574
Best validation accuracy  0
test acc :0.5692307692307692
EPOCH  2
Train : Loss: 0.2237, Train error: 0.3740, Train acc : 0.4712328767123288
Val acc :0.5901639344262296
Best validation accuracy  0
test acc :0.3736263736263736
EPOCH  3
Train : Loss: 0.2163, Train error: 0.3430, Train acc : 0.544987146529563
Val acc :0.6928104575163399
Best validation accuracy  0
test acc :0.5789473684210527
EPOCH  4
Train : Loss: 0.2235, Train error: 0.3547, Train acc : 0.5246753246753247
Val acc :0.6933333333333332
Best validation accuracy  0
test acc :0.6578947368421053
EPOCH  5
Train : Loss: 0.2196, Train error: 0.3624, Train acc : 0.519280205655527
Val acc :0.6842105263157894
Best validation accuracy  0
test acc :0.6
EPOCH  6
Train : Loss: 

Val acc :0.5734265734265734
Best validation accuracy  0.691358024691358
test acc :0.6324786324786326
EPOCH  46
Train : Loss: 0.1433, Train error: 0.2016, Train acc : 0.7709251101321586
Val acc :0.5694444444444444
Best validation accuracy  0.691358024691358
test acc :0.7567567567567567
EPOCH  47
Train : Loss: 0.1459, Train error: 0.2171, Train acc : 0.752212389380531
Val acc :0.5774647887323943
Best validation accuracy  0.691358024691358
test acc :0.6262626262626262
EPOCH  48
Train : Loss: 0.1471, Train error: 0.1996, Train acc : 0.7736263736263737
Val acc :0.5844155844155844
Best validation accuracy  0.691358024691358
test acc :0.7731092436974789
EPOCH  49
Train : Loss: 0.1429, Train error: 0.1841, Train acc : 0.7884187082405345
Val acc :0.5119999999999999
Best validation accuracy  0.691358024691358
test acc :0.717391304347826


In [9]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
device = torch.device("cuda")
model=Attention()
# why Mil
# why attention mil
# why modality specific embedding 
# why non-local
# refere to figures
best_state=torch.load(modelname[-1])
model.load_state_dict(best_state)
model.to(device)
model.eval()
test_loss = 0.
correct = 0
total = 0
y =[]
ypred = []
test_error = 0.
for batch_idx, (data, label) in enumerate(test_loader):
    bag_label = label[0]
    instance_labels = label[1]
    data, bag_label = data.cuda(), bag_label.cuda()
    data, bag_label = Variable(data), Variable(bag_label)
    loss = model.calculate_objective(data.float(), bag_label.float())
    test_loss += loss.item()
    error, predicted_label = model.calculate_classification_error(data.float(), bag_label.float())
    test_error += error
    predicted_label=predicted_label.squeeze(1)
    ypred.extend(predicted_label.tolist())
    y.extend(bag_label.tolist())
    
acc=accuracy_score(y,ypred)
# print(y,ypred)
tn, fp, fn, tp = confusion_matrix(y,ypred).ravel()
f1score=f1_score(y,ypred)
precision=precision_score(y,ypred)
recall=recall_score(y,ypred)
roc=roc_auc_score(y,ypred)
specificity=tn/(tn+fp)
print(acc,f1score,precision,recall,roc,specificity)

for i in modelname:
    print(i)
    model=Attention()
    # best_state=torch.load("../best-models/Attention_MIL_best(stress)_F1_91.pth")
    best_state=torch.load(i)
    model.load_state_dict(best_state)
    model.to(device)
    model.eval()
    test_loss = 0.
    correct = 0
    total = 0
    y =[]
    ypred = []
    test_error = 0.
    for batch_idx, (data, label) in enumerate(test_loader):
        bag_label = label[0]
        instance_labels = label[1]
        data, bag_label = data.cuda(), bag_label.cuda()
        data, bag_label = Variable(data), Variable(bag_label)
        loss = model.calculate_objective(data.float(), bag_label.float())
        test_loss += loss.item()
        error, predicted_label = model.calculate_classification_error(data.float(), bag_label.float())
        test_error += error
        predicted_label=predicted_label.squeeze(1)
        ypred.extend(predicted_label.tolist())
        y.extend(bag_label.tolist())

    acc=accuracy_score(y,ypred)
    # print(y,ypred)
    tn, fp, fn, tp = confusion_matrix(y,ypred).ravel()
    f1score=f1_score(y,ypred)
    precision=precision_score(y,ypred)
    recall=recall_score(y,ypred)
    roc=roc_auc_score(y,ypred)
    specificity=tn/(tn+fp)
    print(acc,f1score,precision,recall,roc,specificity)


/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.


0.7796610169491526 0.7903225806451614 0.7424242424242424 0.8448275862068966 0.7807471264367816 0.7166666666666667
./saved_models/best_model_MIL_raw_cs_nl_epoch_16.pth


/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.


0.7542372881355932 0.7972027972027971 0.6705882352941176 0.9827586206896551 0.7580459770114942 0.5333333333333333
./saved_models/best_model_MIL_raw_cs_nl_epoch_21.pth


/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.


...

0.7627118644067796 0.7971014492753623 0.6875 0.9482758620689655 0.7658045977011494 0.5833333333333334
./saved_models/best_model_MIL_raw_cs_nl_epoch_33.pth


/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.


0.7711864406779662 0.7768595041322314 0.746031746031746 0.8103448275862069 0.7718390804597701 0.7333333333333333
./saved_models/best_model_MIL_raw_cs_nl_epoch_36.pth


/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.


0.7796610169491526 0.7903225806451614 0.7424242424242424 0.8448275862068966 0.7807471264367816 0.7166666666666667


## Window-1

In [10]:
device = torch.device("cuda")
model=Attention()
best_state=torch.load(modelname[-1])
model.load_state_dict(best_state)
model.to(device)
model.eval()
testdataset = load_data('../Participant_wise_physio_change1/','test')
test_loader = data_utils.DataLoader(Stammering(train="test"),worker_init_fn=seed_worker,batch_size=1,shuffle=True)
attn_weights=[]
test_loss = 0.
correct = 0
total = 0
y =[]
ypred = []
test_error = 0.
for batch_idx, (data, label) in enumerate(test_loader):
    bag_label = label[0]
    instance_labels = label[1]
    data, bag_label = data.cuda(), bag_label.cuda()
    data, bag_label = Variable(data), Variable(bag_label)
    loss = model.calculate_objective(data.float(), bag_label.float())
    test_loss += loss.item()
    error, predicted_label = model.calculate_classification_error(data.float(), bag_label.float())
    test_error += error
    predicted_label=predicted_label.squeeze(1)
    ypred.extend(predicted_label.tolist())
    y.extend(bag_label.tolist())
acc=accuracy_score(y,ypred)
tn, fp, fn, tp = confusion_matrix(y,ypred).ravel()
f1score=f1_score(y,ypred)
precision=precision_score(y,ypred)
recall=recall_score(y,ypred)
roc=roc_auc_score(y,ypred)
specificity=tn/(tn+fp)
# print(best_model)
print(acc,f1score,precision,recall,roc,specificity)

/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.


0.7796610169491526 0.7936507936507937 0.7352941176470589 0.8620689655172413 0.7810344827586206 0.7


## Window-2

In [11]:
device = torch.device("cuda")
model=Attention()
best_state=torch.load(modelname[-1])
model.load_state_dict(best_state)
model.to(device)
model.eval()
testdataset = load_data('../Participant_wise_physio_change2/','test')
test_loader = data_utils.DataLoader(Stammering(train="test"),worker_init_fn=seed_worker,batch_size=1,shuffle=True)
attn_weights=[]
test_loss = 0.
correct = 0
total = 0
y =[]
ypred = []
test_error = 0.
for batch_idx, (data, label) in enumerate(test_loader):
    bag_label = label[0]
    instance_labels = label[1]
    data, bag_label = data.cuda(), bag_label.cuda()
    data, bag_label = Variable(data), Variable(bag_label)
    loss = model.calculate_objective(data.float(), bag_label.float())

    test_loss += loss.item()
    error, predicted_label = model.calculate_classification_error(data.float(), bag_label.float())
    test_error += error
    predicted_label=predicted_label.squeeze(1)
    ypred.extend(predicted_label.tolist())
    y.extend(bag_label.tolist())
acc=accuracy_score(y,ypred)
tn, fp, fn, tp = confusion_matrix(y,ypred).ravel()
f1score=f1_score(y,ypred)
precision=precision_score(y,ypred)
recall=recall_score(y,ypred)
roc=roc_auc_score(y,ypred)
specificity=tn/(tn+fp)
# print(best_model)
print(acc,f1score,precision,recall,roc,specificity)

/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:40: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
/home/hsharm04/.conda/envs/physio/lib/python3.7/site-packages/ipykernel_launcher.py:41: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.


0.7796610169491526 0.7868852459016394 0.75 0.8275862068965517 0.7804597701149425 0.7333333333333333


In [12]:
modelname[-1]

'./saved_models/best_model_MIL_raw_cs_nl_epoch_36.pth'